In [40]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [70]:
df = pd.read_csv("Baku.csv", sep=',')
df.columns = ['name','datetime','tempmax','tempmin','temp','feelslikemax','feelslikemin','feelslike','dew','humidity','precip','precipprob','precipcover','preciptype','snow','snowdepth','windgust','windspeed','winddir','sealevelpressure','cloudcover','visibility','solarradiation','solarenergy','uvindex','severerisk','sunrise','sunset','moonphase','conditions','description','icon','stations']

In [71]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')

In [72]:
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year

In [73]:
df.drop(['name','sunrise', 'sunset', 'description', 'stations', 'icon', 'moonphase', 'datetime', 'preciptype'], axis=1, inplace=True)

In [74]:
df

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,day,month,year
0,17.5,7.8,12.0,17.5,6.6,11.7,7.4,76.0,0.0,0,...,41.6,8.9,196.6,17.0,8,10,Partially cloudy,20,3,2023
1,14.0,7.7,10.7,14.0,5.6,10.2,7.6,81.8,0.0,0,...,62.4,8.7,231.7,20.0,8,10,Partially cloudy,21,3,2023
2,15.6,8.9,11.4,15.6,5.3,10.3,4.4,63.9,0.0,0,...,92.9,10.0,212.1,18.5,8,10,Overcast,22,3,2023
3,22.0,6.0,12.3,22.0,2.9,11.4,5.0,67.4,0.0,0,...,51.3,10.0,245.9,21.3,8,10,Partially cloudy,23,3,2023
4,17.0,9.0,12.3,17.0,5.9,11.6,7.4,75.4,0.0,0,...,64.3,8.5,221.8,19.0,7,10,Partially cloudy,24,3,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,13.3,7.9,10.2,13.3,4.9,9.0,6.6,79.9,0.0,0,...,57.2,8.4,220.3,19.0,8,10,Partially cloudy,15,3,2024
362,12.0,7.0,8.8,12.0,3.0,6.7,5.6,81.2,0.3,100,...,60.5,8.2,222.5,19.3,8,10,"Rain, Partially cloudy",16,3,2024
363,12.0,8.0,9.6,12.0,5.0,8.2,5.5,75.9,0.1,100,...,89.4,10.0,111.1,9.6,4,10,"Rain, Partially cloudy",17,3,2024
364,15.0,9.0,11.3,15.0,6.2,10.4,6.8,75.3,0.1,100,...,81.3,10.0,182.5,15.8,7,10,"Rain, Partially cloudy",18,3,2024


In [75]:
df["conditions"].unique()

array(['Partially cloudy', 'Overcast', 'Rain, Partially cloudy',
       'Rain, Overcast', 'Clear', 'Rain', 'Snow, Rain, Partially cloudy',
       'Snow, Overcast', 'Snow, Rain, Overcast', 'Snow, Partially cloudy'],
      dtype=object)

In [76]:
df['conditions'].replace(to_replace="Rain, Partially cloudy", value="Rain", inplace=True)
df['conditions'].replace(to_replace="Rain, Overcast", value="Rain", inplace=True)
df['conditions'].replace(to_replace="Snow, Rain, Partially cloudy", value="Snow/Rain", inplace=True)
df['conditions'].replace(to_replace="Snow, Overcast", value="Snow", inplace=True)
df['conditions'].replace(to_replace="Snow, Rain, Overcast", value="Snow/Rain", inplace=True)
df['conditions'].replace(to_replace="Snow, Partially cloudy", value="Snow", inplace=True)

In [77]:
df[110:120]

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,day,month,year
110,37.0,24.0,31.1,37.3,24.0,31.1,17.5,51.2,0.0,0,...,11.2,10.0,349.6,30.4,10,10,Clear,8,7,2023
111,37.0,27.0,31.2,38.6,28.3,32.8,18.9,49.9,0.0,0,...,30.7,10.0,343.2,29.6,10,10,Partially cloudy,9,7,2023
112,30.2,25.0,27.5,34.2,25.0,29.1,21.6,71.0,0.0,0,...,11.0,10.0,346.7,30.0,10,10,Clear,10,7,2023
113,28.1,23.9,26.2,29.0,23.9,26.7,19.2,65.7,0.0,0,...,42.1,10.0,338.5,29.2,9,10,Partially cloudy,11,7,2023
114,28.0,23.0,25.1,29.2,23.0,25.4,18.2,65.9,0.0,0,...,38.6,10.0,343.0,29.3,10,10,Partially cloudy,12,7,2023
115,30.0,22.0,25.9,30.8,22.0,26.1,17.4,60.5,0.0,0,...,10.0,10.0,342.3,29.4,10,10,Clear,13,7,2023
116,30.1,22.0,26.8,30.4,22.0,26.7,16.0,53.5,0.0,0,...,0.0,10.0,343.1,29.4,10,10,Clear,14,7,2023
117,35.0,22.2,28.4,35.7,22.2,28.6,17.1,53.1,0.0,0,...,7.9,10.0,334.5,28.7,10,10,Clear,15,7,2023
118,29.0,24.8,26.3,30.8,24.8,26.8,20.1,69.2,0.4,100,...,36.3,10.0,230.0,19.9,8,10,Rain,16,7,2023
119,27.0,20.0,23.4,27.7,20.0,23.4,18.1,73.3,103.5,100,...,55.6,9.9,67.3,5.9,5,10,Rain,17,7,2023


In [78]:
df["conditions"].unique()

array(['Partially cloudy', 'Overcast', 'Rain', 'Clear', 'Snow/Rain',
       'Snow'], dtype=object)

In [79]:
label_encoder = LabelEncoder()

# Применение Label Encoding к столбцу 'conditions'
df['conditions'] = label_encoder.fit_transform(df['conditions'])

In [80]:
df["conditions"].unique()

array([2, 1, 3, 0, 5, 4])

In [81]:
X = df.iloc[:,:-1].values 
Y = df.iloc[:,-1:].values 

In [82]:
# Splitting dataset into train and test set   
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 1/3, random_state = 0) 

In [92]:
# Для избежания предупреждения DataConversionWarning, преобразовываем Y_train в одномерный массив
Y_train = Y_train.ravel()

In [88]:
model = KNeighborsClassifier( n_neighbors = 3 ) 
model.fit( X_train, Y_train )

KNeighborsClassifier(n_neighbors=3)

In [89]:
Y_pred = model.predict( X_test ) 

In [90]:
correctly_classified = 0

In [91]:
count = 0      
for count in range( np.size( Y_pred ) ) :
    if Y_test[count] == Y_pred[count] :
        correctly_classified = correctly_classified + 1
              
        count = count + 1
          
print("Accuracy on test set by our model:", (correctly_classified / count ) * 100) 

Accuracy on test set by our model: 87.70491803278688
